In [20]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [21]:
import xgboost as xgb
import lightgbm as gbm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

In [22]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score, classification_report, confusion_matrix, make_scorer

from functools import partial

In [23]:
train = pd.read_parquet('../data/processed/train.parquet')
test = pd.read_parquet('../data/processed/test.parquet')

In [24]:
metric = partial(f1_score, average='macro')
score = make_scorer(f1_score, average='macro')

In [26]:
from imblearn.over_sampling import SMOTE
resampler = SMOTE(random_state=0, k_neighbors=3)
X, y = train.loc[:, ~train.columns.isin(['revenue', "id"])], train.revenue

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=42)

In [28]:
import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

import optuna

In [29]:
# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = lgb.Dataset(train_x, label=train_y)

    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy

In [30]:
def objective_challenge(trial):

    param = {
        "n_estimators":trial.suggest_int("n_estimators", 300, 500, step=50),
        "objective": "binary",
        "random_state": 42,
        "verbosity": 0,
        "boosting_type": "goss",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "max_depth": trial.suggest_int("max_depth", 8,25, step=3),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
    }

    fit_params = {
        "eval_set":[(X_train, y_train), (X_val, y_val)], 
        "early_stopping_rounds":15,
        "eval_metric":[lambda y_true, y_pred: ("F1_MACRO", metric(y_true, np.round(y_pred)), True)],
        "feature_name":X_train.columns.tolist(),
        "categorical_feature":["Month", "TrafficType", "Browser", "SpecialDay", "OperatingSystems", "Region", "VisitorType", "Weekend"]
    }
    
    gbm = lgb.LGBMClassifier(**param, verbose=0)
    
    # In a better scenario, I might use this code.
#     f1_cross = cross_val_score(
#         gbm, X_train, y_train, 
#         cv=StratifiedKFold(3), 
#         scoring=score,
#         fit_params=fit_params
#     )
#     return np.mean(f1_cross)

    gbm.fit(X_train, y_train, **fit_params)
    
    return metric(y_val, gbm.predict(X_val))

In [31]:
study = optuna.create_study(direction="maximize")
study.optimize(objective_challenge, n_trials=120)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-03-08 12:34:46,634] A new study created in memory with name: no-name-ddd80cbb-9996-462e-960a-7b9a835b56be
/home/jloscalzo/.pyenv/versions/default_3.7.9/lib/python3.7/site-packages/optuna/distributions.py:563: UserWarning: The distribution is specified by [8, 25] and step=3, but the range is not divisible by `step`. It will be replaced by [8, 23].
  low=low, old_high=old_high, high=high, step=step
/home/jloscalzo/.pyenv/versions/default_3.7.9/lib/python3.7/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/jloscalzo/.pyenv/versions/default_3.7.9/lib/python3.7/site-packages/lightgbm/basic.py:1555: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Browser', 'Month', 'OperatingSystems', 'Region', 'SpecialDay', 'TrafficType', 'VisitorType', 'Weekend']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Warning] lambda_l2 is set=9.151875734603427e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.151875734603427e-07
[LightGBM] [Warning] verbosity is set=0, verbose=0 will be ignored. Current value: verbosity=0
[LightGBM] [Warning] lambda_l1 is set=6.133071198921238e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6.133071198921238e-07
[LightGBM] [Warning] lambda_l2 is set=9.151875734603427e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.151875734603427e-07
[LightGBM] [Warning] verbosity is set=0, verbose=0 will be ignored. Current value: verbosity=0
[LightGBM] [Warning] lambda_l1 is set=6.133071198921238e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6.133071198921238e-07
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000531 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


/home/jloscalzo/.pyenv/versions/default_3.7.9/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/jloscalzo/.pyenv/versions/default_3.7.9/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] lambda_l2 is set=9.151875734603427e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.151875734603427e-07
[LightGBM] [Warning] verbosity is set=0, verbose=0 will be ignored. Current value: verbosity=0
[LightGBM] [Warning] lambda_l1 is set=6.133071198921238e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6.133071198921238e-07
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	training's binary_logloss: 0.37714	training's F1_MACRO: 0.458158	valid_1's binary_logloss: 0.386198	valid_1's F1_MACRO: 0.458219
Training until validation scores don't improve for 15 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	training's binary_logloss: 0.341294	training's F1_MACRO: 0.458158	valid_1's binary_logloss: 0.357678	valid_1's F1_MACRO: 0.458219
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	training's binary_logloss: 0.314111	training's F1_MACRO: 0.458158	val

[I 2021-03-08 12:34:50,922] Trial 0 finished with value: 0.807009447544516 and parameters: {'n_estimators': 300, 'lambda_l1': 6.133071198921238e-07, 'lambda_l2': 9.151875734603427e-07, 'max_depth': 20, 'num_leaves': 213}. Best is trial 0 with value: 0.807009447544516.


...

[LightGBM] [Warning] lambda_l2 is set=0.9451192336246578, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.9451192336246578
[LightGBM] [Warning] verbosity is set=0, verbose=0 will be ignored. Current value: verbosity=0
[LightGBM] [Warning] lambda_l1 is set=7.247105025012235e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.247105025012235e-06
[LightGBM] [Warning] lambda_l2 is set=0.9451192336246578, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.9451192336246578
[LightGBM] [Warning] verbosity is set=0, verbose=0 will be ignored. Current value: verbosity=0
[LightGBM] [Warning] lambda_l1 is set=7.247105025012235e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=7.247105025012235e-06
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000536 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] lambda_l2 is s

[I 2021-03-08 12:42:40,313] Trial 118 finished with value: 0.7973458964115503 and parameters: {'n_estimators': 450, 'lambda_l1': 7.247105025012235e-06, 'lambda_l2': 0.9451192336246578, 'max_depth': 20, 'num_leaves': 239}. Best is trial 42 with value: 0.8212005983952129.


[LightGBM] [Warning] lambda_l2 is set=5.012064810607244e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.012064810607244e-06
[LightGBM] [Warning] verbosity is set=0, verbose=0 will be ignored. Current value: verbosity=0
[LightGBM] [Warning] lambda_l1 is set=9.730822454156799e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.730822454156799e-08
[LightGBM] [Warning] lambda_l2 is set=5.012064810607244e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.012064810607244e-06
[LightGBM] [Warning] verbosity is set=0, verbose=0 will be ignored. Current value: verbosity=0
[LightGBM] [Warning] lambda_l1 is set=9.730822454156799e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.730822454156799e-08
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000774 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] la

[I 2021-03-08 12:42:42,643] Trial 119 finished with value: 0.8140899260893355 and parameters: {'n_estimators': 400, 'lambda_l1': 9.730822454156799e-08, 'lambda_l2': 5.012064810607244e-06, 'max_depth': 17, 'num_leaves': 244}. Best is trial 42 with value: 0.8212005983952129.


Number of finished trials: 120
Best trial:
  Value: 0.8212005983952129
  Params: 
    n_estimators: 400
    lambda_l1: 8.379812902948931e-07
    lambda_l2: 9.52501649951279e-05
    max_depth: 20
    num_leaves: 208


In [94]:
trial.params

{'n_estimators': 400,
 'lambda_l1': 0.0016108530283158262,
 'lambda_l2': 8.609100631927998,
 'max_depth': 17,
 'num_leaves': 23}

In [33]:
trial.params

{'n_estimators': 400,
 'lambda_l1': 8.379812902948931e-07,
 'lambda_l2': 9.52501649951279e-05,
 'max_depth': 20,
 'num_leaves': 208}

In [87]:
# trial.distributions

In [29]:
# optuna.visualization.plot_optimization_history(study)

In [34]:
list(map(lambda t: t.values, study.get_trials()))

[[0.807009447544516],
 [0.8003705845248108],
 [0.81677257483596],
 [0.8050942773517227],
 [0.8097304744682683],
 [0.807009447544516],
 [0.8113686902075627],
 [0.8197518825622048],
 [0.81299365072143],
 [0.7978930316994833],
 [0.8077621635953864],
 [0.8086079457961195],
 [0.7986079809094524],
 [0.788271832071769],
 [0.8046156880230484],
 [0.8042493152596779],
 [0.8100015491329731],
 [0.7957248274749901],
 [0.8105430787449364],
 [0.8204229930775999],
 [0.8025320075674249],
 [0.7980331427791445],
 [0.8037845517954016],
 [0.8037845517954016],
 [0.8138649520526757],
 [0.7915823301729341],
 [0.8102724132516879],
 [0.7978930316994833],
 [0.8025320075674249],
 [0.7977122616384362],
 [0.8102724132516879],
 [0.8122001569438748],
 [0.8127531937550225],
 [0.798226217267668],
 [0.8001653746770025],
 [0.7990971781933142],
 [0.8023143873375257],
 [0.803619203206487],
 [0.8033952726380711],
 [0.8014491465246933],
 [0.7944852126579516],
 [0.7986079809094524],
 [0.8212005983952129],
 [0.8025320075674249

In [35]:
study.get_trials()[18]

FrozenTrial(number=18, values=[0.8105430787449364], datetime_start=datetime.datetime(2021, 3, 8, 12, 36, 12, 46760), datetime_complete=datetime.datetime(2021, 3, 8, 12, 36, 16, 727691), params={'n_estimators': 400, 'lambda_l1': 1.278535014544274e-05, 'lambda_l2': 9.693557882425107, 'max_depth': 11, 'num_leaves': 193}, distributions={'n_estimators': IntUniformDistribution(high=500, low=300, step=50), 'lambda_l1': LogUniformDistribution(high=10.0, low=1e-08), 'lambda_l2': LogUniformDistribution(high=10.0, low=1e-08), 'max_depth': IntUniformDistribution(high=23, low=8, step=3), 'num_leaves': IntUniformDistribution(high=256, low=2, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=18, state=TrialState.COMPLETE, value=None)

In [36]:
fit_params = {
    "eval_set":[(X_train, y_train), (X_val, y_val)], 
    "early_stopping_rounds":15,
    "eval_metric":[lambda y_true, y_pred: ("F1_MACRO", metric(y_true, np.round(y_pred)), True)],
    "feature_name":X_train.columns.tolist(),
    "categorical_feature":["Month", "TrafficType", "Browser", "SpecialDay", "OperatingSystems", "Region", "VisitorType", "Weekend"]
}
    
gbm = lgb.LGBMClassifier(**trial.params, verbose=0)
    
gbm.fit(X, y, **fit_params)

/home/jloscalzo/.pyenv/versions/default_3.7.9/lib/python3.7/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/jloscalzo/.pyenv/versions/default_3.7.9/lib/python3.7/site-packages/lightgbm/basic.py:1555: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Browser', 'Month', 'OperatingSystems', 'Region', 'SpecialDay', 'TrafficType', 'VisitorType', 'Weekend']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Warning] lambda_l1 is set=8.379812902948931e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.379812902948931e-07
[LightGBM] [Warning] lambda_l2 is set=9.52501649951279e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.52501649951279e-05
[LightGBM] [Warning] lambda_l1 is set=8.379812902948931e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.379812902948931e-07
[LightGBM] [Warning] lambda_l2 is set=9.52501649951279e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.52501649951279e-05
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000776 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


/home/jloscalzo/.pyenv/versions/default_3.7.9/lib/python3.7/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/jloscalzo/.pyenv/versions/default_3.7.9/lib/python3.7/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] lambda_l1 is set=8.379812902948931e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.379812902948931e-07
[LightGBM] [Warning] lambda_l2 is set=9.52501649951279e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.52501649951279e-05
[LightGBM] [Warning] lambda_l1 is set=8.379812902948931e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.379812902948931e-07
[LightGBM] [Warning] lambda_l2 is set=9.52501649951279e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.52501649951279e-05
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's binary_logloss: 0.377293	valid_0's F1_MACRO: 0.458158	valid_1's binary_logloss: 0.37694	valid_1's F1_MACRO: 0.458219
Training until validation scores don't improve for 15 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's binary_logloss: 0.341832	valid_0's F1_MACRO: 0.458158	valid_1's binary_logloss: 0.340227	v

LGBMClassifier(lambda_l1=8.379812902948931e-07, lambda_l2=9.52501649951279e-05,
               max_depth=20, n_estimators=400, num_leaves=208, verbose=0)

In [37]:
metric(y, gbm.predict(X))

0.9993347953807932

In [38]:
ids = test.id
test.drop(columns=["id"], inplace=True)

In [39]:
result = gbm.predict(test)

In [40]:
from datetime import datetime
get_date = lambda : datetime.now().strftime("%Y%m%d_%H%M%S")

In [41]:
pd.DataFrame({"id":ids, "revenue":result}).to_csv(f'../data/results/{get_date()}_lgb_opt.csv', index=False)

In [42]:
gbm._Booster.save_model(f'../data/results/{get_date()}_lgb_opt_model.txt')